### 작성일 : 19/08/06
### 목적 : 변수 별 의미 파악을 통해 (동일한 사람을 grouping 할수있는 힌트를 얻자) <- 하지만 현실은 삽질뿐...

In [2]:
# load libraies & data import
library(data.table)
library(ggplot2)
library(dplyr)
options(repr.plot.width = 5, repr.plot.height = 3)
options(scipen = 100)
library(ggpubr)
library(magrittr)

tr <- fread("/Users/jeong/downloads/IEEE/input/train_transaction.csv")
te <- fread("/Users/jeong/downloads/IEEE/input/test_transaction.csv")

#### 1.  (가설) Card1은 카드 상품(ex. 삼성 더오 신용카드)일 가능성

In [19]:
tr$card23456 <- paste0(tr$card2, tr$card3, tr$card4, tr$card5, tr$card6) # card2~6을 붙여서 하나의 컬럼으로 재생성
ex <- tr[, .(unique_card2 = uniqueN(card2),
             uniq_card3 = uniqueN(card3),
             unique_card4 = uniqueN(card4),
             uniq_card5 = uniqueN(card5),
             unique_card6 = uniqueN(card6),
            unique_card23456 = uniqueN(card23456)) , by= card1] # card1별로 unique count 세기
ex <- ex[order(-unique_card23456)]
summary(ex$unique_card23456)

nrow(ex[unique_card23456==1]) / nrow(ex) # 전체 card1 중 card2~6조합이 한가지인 경우의 비율을 구함

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  1.000   1.000   1.000   1.099   1.000   4.000 

[1] 0.9081384

In [8]:
ex <- tr[, .(uniq_card1 = uniqueN(card1)
            ) , by= c('card3', 'card5')]
ex <- ex[order(-uniq_card1)]
head(ex, 5) # card3, card5로 콤비네이션 결과 unique card1 값들이 많이 나왔습니다..

card3,card5,uniq_card1
<dbl>,<dbl>,<int>
150,226,5216
150,224,1784
150,195,948
NA,NA,671
150,117,646


- card1은 card3 x card5의 콤비네이션 보다는 별개의 컬럼이라고 판단(위 표의 uniq_card1 참조)
- (가설) 약 90%의 card1은 card2~card6가 하나의 값을 가지는걸로 보아 카드 상품(ex 삼성 더오 신용카드, 신한 xx 체크카드 등)라고 추측
    - 이를 뒷받침하는 추가 데이터 분석 결과가 필요

#### ??

앞선 card1 별로 count와 fraud_ratio를 구하고, Fraud_ratio가 높은 친구들을 관찰

In [20]:
ex <- tr[, .(cnt = .N,
            prop = mean(isFraud)), by= card1] # card1별로 value count 및 fraud ratio를 구하자
ex <- ex[order(-cnt)]
head(ex, 20)

card1,cnt,prop
<int>,<int>,<dbl>
7919,14932,0.00750067
9500,14162,0.03728287
15885,10361,0.04285301
17188,10344,0.02687548
15066,7945,0.03939585
12695,7091,0.02834579
12544,6773,0.02155618
6019,6771,0.04342047
2803,6141,0.01188731


card1 == 9633의 경우 fraud ratio가 17%에 육박..

In [30]:
ex <- tr[card1==9633]
ex$day <- round( ex$TransactionDT / 60 / 60 / 42, 0)
ex$hour <- round( ex$TransactionDT / 60 / 60 ,0) %% 24

In [49]:
ex[801:850, c('isFraud', 'TransactionDT', 'TransactionAmt','day','hour', 'ProductCD','D1', 'D2',
       'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15')]

isFraud,TransactionDT,TransactionAmt,day,hour,ProductCD,D1,D2,D3,D4,⋯,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15
<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2051733,38.012,14,18,C,6,6,0,0,⋯,0,0,358.708344,0.708333,0,NA,0,NA,360,0
0,2052045,24.136,14,18,C,0,NA,NA,0,⋯,0,NA,NA,NA,0,NA,0,NA,NA,0
0,2052563,97.944,14,18,C,0,NA,NA,0,⋯,0,NA,NA,NA,0,NA,0,0,NA,0
0,2056811,27.401,14,19,C,0,NA,NA,0,⋯,0,NA,NA,NA,0,NA,0,0,NA,0
0,2057038,10.261,14,19,C,0,NA,NA,0,⋯,0,NA,NA,NA,0,NA,0,NA,NA,0
0,2059516,123.384,14,20,C,0,NA,NA,0,⋯,0,0,NA,NA,0,NA,0,NA,NA,0
0,2060216,144.796,14,20,C,0,0,0,0,⋯,0,NA,NA,NA,0,NA,0,0,0,0
0,2061649,17.723,14,21,C,0,0,0,0,⋯,0,NA,NA,NA,0,NA,0,NA,NA,0
0,2061813,23.903,14,21,C,33,33,33,106,⋯,106,33,NA,NA,0,NA,106,0,0,106
